In [22]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [23]:
# Loading data
pd.set_option('display.max_rows', 25)

tr = pd.read_csv('../input/titanic/train.csv')
tes = pd.read_csv('../input/titanic/test.csv')

train_data = tr.copy()
test_data = tes.copy()

In [24]:
# Preprocessing

a = {'male': 1, 'female': 0}
b = {'C' : 1, 'S': 2, 'Q': 3}


train_data['Sex'] = train_data['Sex'].map(a)
train_data['Embarked'] = train_data['Embarked'].map(b)
train_data = train_data.drop(['PassengerId','Ticket', 'Cabin'], axis=1)

test_data['Sex'] = test_data['Sex'].map(a)
test_data['Embarked'] = test_data['Embarked'].map(b)
test_data = test_data.drop(['PassengerId','Ticket', 'Cabin'], axis=1)

train_data = train_data.dropna()
test_data = test_data.dropna()


train_data['Embarked'] = train_data['Embarked'].astype(int)
train_data['Fare'] = train_data['Fare'].round()

In [25]:

new_name_list = []
name_list = train_data['Name']

for x in name_list:
    if 'Mr.' in x:
        new_name_list.append('1')
    elif 'Mrs.' in x:
        new_name_list.append('2')
    elif 'Miss.' in x:
        new_name_list.append('3')
    elif 'Rev.' in x:
        new_name_list.append('4')
    elif 'Master.' in x:
        new_name_list.append('5')
    elif 'Dr.' in x:
        new_name_list.append('6')
    elif 'Don.' in x:
        new_name_list.append('7')
    else:
        new_name_list.append('0')

train_data['Name'] = new_name_list


In [26]:
s = train_data.copy()
survived_list = s[s['Survived'] == 1]
d = train_data.copy()
dead_list = d[d['Survived'] == 0]
new_dead_list = dead_list.iloc[:len(survived_list),:]
balanced_train_data = pd.concat([survived_list,dead_list])
balanced_train_data
    

In [27]:
# One-Hot encoding for Name, Embarked columns
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split



name_train_dummies = pd.get_dummies(data=balanced_train_data['Name'])
emb_train_dummies = pd.get_dummies(data=balanced_train_data['Embarked'])


balanced_train_data = balanced_train_data.drop(["Name","Embarked"], axis=1)


balanced_train_data = pd.concat([balanced_train_data, name_train_dummies, emb_train_dummies], axis=1)


balanced_train_data = balanced_train_data.rename(columns = {'0':'NoStatus', '1':'Mr', '2':'Mrs', '3':'Miss', '4':'Rev', '5':'Master'
                                    , '6':'Dr', '7':'Don'})


y = balanced_train_data.Survived
X = balanced_train_data.drop(['Survived'], axis=1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

X_valid

In [28]:
# Model & Training - Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


rand_forest = RandomForestClassifier(n_estimators = 100)
rand_forest.fit(X_train, y_train)
preds = rand_forest.predict(X_valid)
score = accuracy_score(y_valid, preds)
print(score)

In [31]:
score_matrix['Random Forest Classifier'] = score

score_matrix

In [32]:
# Model & Training - Naive Bayes
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()
gnb.fit(X_train, y_train)
gnb_preds = gnb.predict(X_valid)
nb = [accuracy_score(y_valid, gnb_preds)]
score_matrix['NaiveBayes'] = nb

In [33]:
# Model & Training - Logistic Regression
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
logreg_preds = logreg.predict(X_valid)
lr = [accuracy_score(y_valid, logreg_preds)]
score_matrix['LogisticREgression'] = lr
score_matrix

In [37]:
# Model & Training - K Nearest Neighbor
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=100)
knn.fit(X_train, y_train)
knn_preds = knn.predict(X_valid)
knn_mat = accuracy_score(y_valid, knn_preds)
score_matrix['KNN'] = knn_mat


In [38]:
# Model & Training - Support Vector Machine
from sklearn import svm
svm = svm.SVC()
svm.fit(X_train, y_train)
svm_preds = svm.predict(X_valid)
svm_mat = accuracy_score(y_valid, svm_preds)
score_matrix['SVM'] = svm_mat


In [39]:
score_matrix